In [15]:
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import pandas as pd
import time
import seaborn as sns
from sklearn import metrics
from sklearn.decomposition import PCA
from collections import Counter

In [246]:
datasetPath = 'Diabetes_Data_1999_2008.csv'

In [247]:
data = pd.read_csv(datasetPath)

In [251]:
replaceDict = {'[0-10)' : 5,
'[10-20)' : 15,
'[20-30)' : 25, 
'[30-40)' : 35, 
'[40-50)' : 45, 
'[50-60)' : 55,
'[60-70)' : 65, 
'[70-80)' : 75,
'[80-90)' : 85,
'[90-100)' : 95}

data['age'] = data['age'].apply(lambda x : replaceDict[x])

In [252]:
data.dropna(inplace = True)
data.drop_duplicates(['patient_nbr'], keep = 'first', inplace = True)
data

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,5,?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,15,?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,25,?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,35,?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,45,?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101754,443842016,183087545,Caucasian,Female,75,?,1,1,7,9,...,No,Steady,No,No,No,No,No,Ch,Yes,>30
101755,443842022,188574944,Other,Female,45,?,1,1,7,14,...,No,Up,No,No,No,No,No,Ch,Yes,>30
101756,443842070,140199494,Other,Female,65,?,1,1,7,2,...,No,Steady,No,No,No,No,No,No,Yes,>30
101758,443842340,120975314,Caucasian,Female,85,?,1,1,7,5,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [253]:
print(list(data))

['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted']


In [254]:
high_frequency = ['InternalMedicine', 'Family/GeneralPractice', 'Cardiology', 'Surgery-General', 'Orthopedics', 'Orthopedics-Reconstructive', 'Emergency/Trauma', 'Urology','ObstetricsandGynecology','Psychiatry','Pulmonology ','Nephrology','Radiologist']
low_frequency = ['Surgery-PlasticwithinHeadandNeck','Psychiatry-Addictive','Proctology','Dermatology','SportsMedicine','Speech','Perinatology', 'Neurophysiology','Resident','Pediatrics-Hematology-Oncology','Pediatrics-EmergencyMedicine','Dentistry','DCPTEAM','Psychiatry-Child/Adolescent', 'Pediatrics-Pulmonology','Surgery-Pediatric','AllergyandImmunology','Pediatrics-Neurology','Anesthesiology','Pathology','Cardiology-Pediatric', 'Endocrinology-Metabolism','PhysicianNotFound','Surgery-Colon&Rectal','OutreachServices', 'Surgery-Maxillofacial','Rheumatology','Anesthesiology-Pediatric','Obstetrics','Obsterics&Gynecology-GynecologicOnco']
pediatrics = ['Pediatrics','Pediatrics-CriticalCare','Pediatrics-EmergencyMedicine','Pediatrics-Endocrinology','Pediatrics-Hematology-Oncology', 'Pediatrics-Neurology','Pediatrics-Pulmonology', 'Anesthesiology-Pediatric', 'Cardiology-Pediatric', 'Surgery-Pediatric']
psychic = ['Psychiatry-Addictive', 'Psychology', 'Psychiatry',  'Psychiatry-Child/Adolescent', 'PhysicalMedicineandRehabilitation', 'Osteopath']
neurology = ['Neurology', 'Surgery-Neuro',  'Pediatrics-Neurology', 'Neurophysiology']
surgery = ['Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Maxillofacial', 'Surgery-Plastic', 'Surgery-PlasticwithinHeadandNeck',  'Surgery-Thoracic', 'Surgery-Vascular', 'SurgicalSpecialty', 'Podiatry']
ungrouped = ['Endocrinology','Gastroenterology','Gynecology','Hematology','Hematology/Oncology','Hospitalist','InfectiousDiseases', 'Oncology','Ophthalmology','Otolaryngology','Pulmonology','Radiology']
missing = ['?']

colMedical = []

for val in data['medical_specialty'] :
    if val in pediatrics :
        colMedical.append('pediatrics')
    elif val in psychic :
        colMedical.append('psychic')
    elif val in neurology :
        colMedical.append('neurology')
    elif val in surgery :
        colMedical.append('surgery')
    elif val in high_frequency :
        colMedical.append('high_freq')
    elif val in low_frequency :
        colMedical.append('low_freq')
    elif val in ungrouped :
        colMedical.append('ungrouped')
    elif val in missing :
        colMedical.append('missing')

data['medical_specialty'] = colMedical

In [255]:
def get_diag(diag):
    if '?' in diag: return 'unknown'
    if 'V' in diag:  return 'health_contact'
    if 'E' in diag: return 'injury_poisoning'
    diag_no = int(float(diag))
    if diag_no in range(0, 140): return 'infectious'
    if diag_no in range(140, 240): return 'neoplasms'
    if diag_no == 250: return 'diabetes'
    if diag_no in range(251, 260): return 'endocrine'
    if diag_no in range(240, 280): return 'long_term'
    if diag_no in range(280, 290): return 'blood'
    if diag_no in range(290, 320): return 'mental'
    if diag_no in range(320, 390): return 'nervous'
    if diag_no in range(390, 460): return 'respiratory'
    if diag_no in range(520, 580): return 'digestive'
    if diag_no in range(580, 630): return 'genitourinary'
    if diag_no in range(630, 680): return 'birth'
    if diag_no in range(680, 710): return 'skin'
    if diag_no in range(710, 740): return 'musculoskeletal'
    if diag_no in range(740, 760): return 'congenital'
    if diag_no in range(760, 780): return 'prenital'
    if diag_no in range(780, 800): return 'symptoms'
    if diag_no in range(800, 1000): return 'injury_poisoning'
    return 'other'

In [256]:
data['diag_1'] = data['diag_1'].apply(get_diag)
data['diag_2'] = data['diag_2'].apply(get_diag)
data['diag_3'] = data['diag_3'].apply(get_diag)

In [257]:
data['readmitted'] = data['readmitted'].apply(lambda x : 0 if (x == '>30' or x == 'NO') else 1)
data = data.reset_index()

In [258]:
def get_aic(r):
    if r == 'None':
        return [1,0,0]
    if r == 'Norm':
        return [0,1,0]
    if r == '>7' or r == '>8':
        return [0,0,1]

def get_glu(r):
    if r == 'None':
        return [1,0,0]
    if r == 'Norm':
        return [0,1,0]
    if r == '>200' or r == '>300':
        return [0,0,1]

def get_medicine(m):
    if m == 'No' or m == 'Steady':
        return 0
    if m == 'Up' or m == 'Down':
        return 1

def get_diabetes(d):
    if d == 'Yes':
        return 1
    if d == 'No':
        return 0

def get_ch(c):
    if c == 'Ch':
        return 1
    if c == 'No':
        return 0

data['A1Cresult'] = data['A1Cresult'].apply(get_aic)
data['max_glu_serum'] = data['max_glu_serum'].apply(get_glu)
for m in data.columns[25:48]:
    data[m] = data[m].apply(get_medicine)
data['diabetesMed'] = data['diabetesMed'].apply(get_diabetes)
data['change'] = data['change'].apply(get_ch)

In [259]:
race = np.unique(data['race'])
gender = np.unique(data['gender'])
def get_race(r):
    return (race == r) / 1
def get_gender(g):
    return (gender == g) / 1

data['race'] = data['race'].apply(get_race)
data['gender'] = data['gender'].apply(get_gender)

In [260]:
def logfunc(m):
    return math.log(m+1)

data['num_medications'] = data['num_medications'].apply(logfunc)
data['num_procedures'] = data['num_procedures'].apply(logfunc)
data['num_lab_procedures'] = data['num_lab_procedures'].apply(logfunc)
data['number_diagnoses'] = data['number_diagnoses'].apply(logfunc)
data['number_emergency'] = data['number_emergency'].apply(logfunc)
data['number_inpatient'] = data['number_inpatient'].apply(logfunc)
data['number_outpatient'] = data['number_outpatient'].apply(logfunc)
data['time_in_hospital'] = data['time_in_hospital'].apply(logfunc)

In [261]:
data['total_services'] = data['number_inpatient'] + data['number_outpatient'] + data['number_emergency']
data['change_meds'] = np.sum([data[m] for m in data.columns[25:48]],axis=0)

In [262]:
data.head()

,index,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,...,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,total_services,change_meds
0,0,2278392,8222157,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",5,?,6,25,1,...,0,0,0,0,0,0,0,0,0.000000,0
1,1,149190,55629189,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",15,?,1,1,7,...,0,0,0,0,0,1,1,0,0.000000,1
2,2,64410,86047875,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",25,?,1,1,7,...,0,0,0,0,0,0,1,0,1.791759,0
3,3,500364,82442376,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0]",35,?,1,1,7,...,0,0,0,0,0,1,1,0,0.000000,1
4,4,16680,42519267,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0]",45,?,1,1,7,...,0,0,0,0,0,1,1,0,0.000000,0


In [12]:
def getValuesByGroup(tbl, group):
    grouped = tbl.groupby(group)
    groups = np.unique(tbl[group])
    output = {}
    for g in groups:
        output[g] = grouped.get_group(g)
    return output

In [13]:
pc = getValuesByGroup(data, 'payer_code')
readmitted = len(np.where(data['readmitted'] == 1)[0])
for k in pc:
    print(len(np.where(pc[k]['readmitted'] == 1)[0]) / readmitted, k, len(np.where(pc[k]['readmitted'] == 1)[0])/len(pc[k]))

0.4797393929763229 ? 0.09725219856328318
0.03225806451612903 BC 0.05914918414918415
0.001430160495788972 CH 0.07692307692307693
0.01668520578420467 CM 0.07871064467766117
0.019863340219291276 CP 0.06294058408862034
0.005402828539647227 DM 0.09090909090909091
0.0 FR 0.0
0.0441760686477038 HM 0.06911984087518648
0.29477196885428253 MC 0.09086901146272167
0.027490862863499128 MD 0.07874374146563495
0.0003178134435086604 MP 0.0625
0.00794533608771651 OG 0.07598784194528875
0.0004767201652629906 OT 0.046153846153846156
0.004131574765612585 PO 0.056155507559395246
0.0003178134435086604 SI 0.05405405405405406
0.041792467821388846 SP 0.07857783089333732
0.02240584776736056 UN 0.07503991484832358
0.0007945336087716511 WC 0.042735042735042736
